In [2]:
setwd('~/Documents/python_code/text_mining/hw3/')
X_train = read.csv('X_train.csv')
X_test = read.csv('X_test.csv')
y_train = read.csv('y_train.csv')
y_test = read.csv('y_test.csv')

In [4]:
library('textir')

In [5]:
# Fit LASSO-GAMMA
fit <- mnlm(NULL, y_train, X_train)

In [6]:
term_loading = data.frame('coef'=coef(fit)[2,])
term_loading$absCoef = abs(term_loading$coef)

In [7]:
term_loading = term_loading[order(term_loading$absCoef, decreasing = TRUE),]

In [8]:
head(term_loading, 10)

,coef,absCoef
,<dbl>,<dbl>
garbag,-2.117111,2.117111
wack,-2.032871,2.032871
horribl,-1.962467,1.962467
worst,-1.872887,1.872887
aw,-1.806950,1.806950
terribl,-1.735553,1.735553
wors,-1.675908,1.675908
mediocr,-1.670686,1.670686
suck,-1.661779,1.661779


In [9]:
tail(term_loading, 10)

,coef,absCoef
,<dbl>,<dbl>
week,0,0
went,0,0
west,0,0
white,0,0
win,0,0
word,0,0
worthi,0,0
wow,0,0
york,0,0


In [10]:
# fd <- X_train / rowSums(X_train)
# fd <- data.matrix(fd)
# zd = fd %*% b
# textir package implements the above projection
srp <- srproj(fit, data.matrix(X_train))

# Logistic regression as general linear model
df <- data.frame('Projection'=srp[,1],  'Label'=y_train[,1])
df$Label <- factor(df$Label)  # As categorical
model <- glm(Label ~ Projection, data=df, family='binomial')
summary(model)


Call:
glm(formula = Label ~ Projection, family = "binomial", data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.9859   0.2324   0.3994   0.5819   4.5002  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  1.50996    0.01251  120.71   <2e-16 ***
Projection  12.21484    0.13072   93.44   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 57536  on 58233  degrees of freedom
Residual deviance: 44913  on 58232  degrees of freedom
AIC: 44917

Number of Fisher Scoring iterations: 5


In [11]:
# In sample prediction
glm.probs <- predict(model, type='response')
glm.pred <- ifelse(glm.probs>0.5, 1, 0)
# Accuracy score
sum(glm.pred == y_train) / dim(y_train)[1]

[1] 0.8307003

In [12]:
# Construct test set
test_set = data.frame(Projection=srproj(fit, data.matrix(X_test))[,1],  # Project the test X
                       Label=y_test[,1]
                      )

In [13]:
# Out-of-sample prediction
glm.probs <- predict(model, type='response', newdata = test_set)
glm.pred <- ifelse(glm.probs>0.5, 1, 0)
# Accuracy score
sum(glm.pred == y_train) / dim(y_train)[1]

[1] 0.7510904